In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
os.chdir("/kaggle/working/")
# Any results you write to the current directory are saved as output.

In [ ]:
df_train = pd.read_csv('../input/train.csv',nrows = 50000,parse_dates=["pickup_datetime"])
df_train.head()

In [ ]:
df_train.describe()
df_train.dtypes

In [ ]:
df_train = df_train[(df_train['fare_amount']>0.05) & (df_train.passenger_count>0)]
#df_train = df_train[df_train.passenger_count>0]
df_train.dropna(how = 'any', axis = 'rows',inplace=True)
print('New Size: {}'.format(len(df_train)))
df_test =  pd.read_csv('../input/test.csv',parse_dates=["pickup_datetime"])

In [ ]:
mask = df_train['pickup_longitude'].between(-75, -73)
mask &= df_train['dropoff_longitude'].between(-75, -73)
mask &= df_train['pickup_latitude'].between(40, 42)
mask &= df_train['dropoff_latitude'].between(40, 42)
mask &= df_train['passenger_count'].between(0, 8)
mask &= df_train['fare_amount'].between(0, 250)

df_train = df_train[mask]

In [ ]:
def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295 # Pi/180
    a = 0.5 - np.cos((lat2 - lat1) * p)/2 + np.cos(lat1 * p) * np.cos(lat2 * p) * (1 - np.cos((lon2 - lon1) * p)) / 2
    return 0.6213712 * 12742 * np.arcsin(np.sqrt(a)) # 2*R*asin...

df_train['distance_miles'] = distance(df_train.pickup_latitude, df_train.pickup_longitude,
                                      df_train.dropoff_latitude, df_train.dropoff_longitude)
df_train['year'] = df_train.pickup_datetime.apply(lambda t: t.year)
df_train['hour'] = df_train.pickup_datetime.apply(lambda t: t.hour)
df_test['distance_miles'] = distance(df_test.pickup_latitude, df_test.pickup_longitude,
                                      df_test.dropoff_latitude, df_test.dropoff_longitude)
df_test['hour'] = df_test.pickup_datetime.apply(lambda t: pd.to_datetime(t).hour)
df_test['year'] = df_test.pickup_datetime.apply(lambda t: pd.to_datetime(t).year)

In [ ]:
features = ['year', 'hour', 'distance_miles', 'passenger_count']
X = df_train[features].values
y = df_train['fare_amount'].values
X_test = df_test[features]
df_test.head(5)

In [ ]:
import xgboost as xgb
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split




In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.2)
X_train = xgb.DMatrix(X_train, label=y_train)
X_val = xgb.DMatrix(X_val)
def xgb_eva(max_depth,gamma,colsample_bytree):
    params = {'eval_metric': 'rmse',
              'max_depth': int(max_depth),
              'subsample': 0.8,
              'eta': 0.1,
              'gamma': gamma,
              'colsample_bytree': colsample_bytree}
    cv_result = xgb.cv(params, X_train, num_boost_round=100, nfold=3)
    return -1.0 * cv_result['test-rmse-mean'].iloc[-1]

In [ ]:
xgb_bo = BayesianOptimization(xgb_eva, {'max_depth': (3, 7), 
                                        'gamma': (0, 1),
                                        'colsample_bytree': (0.3, 0.9)})
# Use the expected improvement acquisition function to handle negative numbers
# Optimally needs quite a few more initiation points and number of iterations
xgb_bo.maximize(init_points=3, n_iter=5, acq='ei')

In [ ]:
params = xgb_bo.res['max']['max_params']
params['max_depth'] = int(params['max_depth'])


In [ ]:
# Train a new model with the best parameters from the search
#model2 = xgb.train(params, X, num_boost_round=250)

# Predict on testing and training set
#y_val_pred = model2.predict(X_val)
#y_train_pred = model2.predict(X_train)

# Report testing and training RMSE
#print(np.sqrt(mean_squared_error(y_val, y_val_pred)))
#print(np.sqrt(mean_squared_error(y_train, y_train_pred)))

In [ ]:
model2 = xgb.train(params, xgb.DMatrix(X,label=y), num_boost_round=250)
X_testm = xgb.DMatrix(X_test.values)
y_test = model2.predict(X_testm)


In [ ]:
X_test.head(5)

In [ ]:

sub = pd.DataFrame()
sub['key'] = df_test.key
sub['fare_amount'] = y_test
sub.to_csv('submission.csv',index=False)
#out = pd.DataFrame({'key':X_test.index,'fare_amount':y_test})
#out.to_csv('submission.csv',index=False)
sub